In [1]:
import re
import json
import locale
import requests
import pandas as pd
from mplogger import Logger
from datetime import datetime
from bs4 import BeautifulSoup
from collections import Counter
from html2tei import parse_date

from html2tei import parse_date
from warcio.archiveiterator import ArchiveIterator
from webarticlecurator import WarcCachingDownloader

from origo_specific import get_meta_from_articles_spec as get_meta

## NOTES
- Ezzekkel a galéria cikkekkel mi lesz? https://www.origo.hu/auto/galeria/20220317-volkswagen-t7-multivan-etsi-teszt-2022-galeria.html
- https://www.origo.hu/sport/kozvetites/20211009-magyarorszag-albania-vbselejtezo-elo-kozvetites-puskas-arena-kepek-percrol-percre.html Formátum 3
- Miért rossz az olimpiás? https://www.origo.hu/sport/olimpia/galeria/20210726-gordeszka-latvanyos-kepek.html
- A közvetítésnél mi legyen az sch:name? 

In [4]:
logger = Logger()

2022-03-30 13:51:45,717 INFO: Logging started
2022-03-30 13:51:45,717 INFO: Logging started


In [6]:
# Links:
kozvetites_link = 'https://www.origo.hu/sport/kozvetites/20211202-kezilabda-noi-vilagbajnoksag-magyarorszag-szlovakia-elo-kozvetites.html'
olimpia_link = 'https://www.origo.hu/sport/olimpia/galeria/20210726-gordeszka-latvanyos-kepek.html'
ronaldo_link = 'https://www.origo.hu/sport/futball/20220315-ronaldo-ismet-bizonyithat-kedvenc-ellenfele-ellen-otthon-juthat-tovabb-a-bl-titkos-eselyese.html'
auto_link = 'https://www.origo.hu/auto/20220325-a-forgalommal-szemben-kanyarodott-fel-a-4es-utra-balszerencsejere-pont-ott-voltak-a-rendorok-video.html'
gallery_link = 'https://www.origo.hu/sport/galeria/20220317-a-szexi-luisinha-barosa-oliveira-mclaren-lando-norris-kepek-fotok-galeria.html'

In [3]:
def response_warc_record_gen(warc_filename):
    archive_base = ArchiveIterator(open(warc_filename, 'rb'))
    for rec in archive_base:
        if rec.rec_type == 'response':
            article_url = rec.rec_headers.get_header('WARC-Target-URI')
            raw_html = rec.content_stream().read()
            yield article_url, raw_html

def response_warc_record_gen(warc_filename):
    archive_base = ArchiveIterator(open(warc_filename, 'rb'))
    for rec in archive_base:
        if rec.rec_type == 'response':
            article_url = rec.rec_headers.get_header('WARC-Target-URI')
            raw_html = rec.content_stream().read()
            yield article_url, raw_html

def extract_resp_record_data(resp):
    """Extract response date, WARC ID and raw HTML from a WARC response record"""
    warc_response_date = resp.rec_headers.get_header('WARC-Date')
    if '.' in warc_response_date:
        date_format = '%Y-%m-%dT%H:%M:%S.%fZ'
    else:
        date_format = '%Y-%m-%dT%H:%M:%SZ'
    warc_response_datetime = datetime.strptime(warc_response_date, date_format)
    warc_id = resp.rec_headers.get_header('WARC-Record-ID')
    raw_html = resp.content_stream().read().decode(resp.rec_headers.get_header('WARC-X-Detected-Encoding'))

    return warc_response_datetime, warc_id, raw_html

def from_request(link):
    txt = requests.get(link).text
    soup = BeautifulSoup(txt, 'lxml')
    return soup

In [5]:
<meta name="publish-date" content="2022-03-17" />
<meta name="modified-date" content="2022-03-17" />

SyntaxError: invalid syntax (4218791069.py, line 1)

In [6]:
date = from_request(gallery_link).find('meta', {'name':'publish-date'})['content']
print(date)

2022-03-17


In [7]:
parse_date(date, "%Y-%m-%d")

datetime.datetime(2022, 3, 17, 0, 0)

In [8]:
get_meta(logger, kozvetites_link, from_request(kozvetites_link).find('title'))

https://www.origo.hu/sport/kozvetites/20211202-kezilabda-noi-vilagbajnoksag-magyarorszag-szlovakia-elo-kozvetites.html ARTICLE FORMAT UNACCOUNTED FOR


defaultdict(<function html2tei.tei_utils.tei_defaultdict.<locals>.<lambda>()>,
            {'sch:url': 'https://www.origo.hu/sport/kozvetites/20211202-kezilabda-noi-vilagbajnoksag-magyarorszag-szlovakia-elo-kozvetites.html',
             'sch:name': None})

In [7]:
from_request(kozvetites_link).find('div', class_='sportonline_header').find('')

<div class="sportonline_header" id="article-head">
<div id="sportonline_cikk_header">
<div class="so_header_timeline_datas">
<div class="so_extra_data_1"></div>
<div class="so_header_team_datas container clearfix">
<div class="so_header_team so_header_team_left clearfix">
<div class="so_team_logo">
<img alt="" height="84" src="" width="84"/>
</div>
<div class="so_team_name"><span class="so_team_shortname"></span></div>
<div class="so_team_point"></div>
</div>
<div class="so_header_team so_header_team_right clearfix">
<div class="so_team_point"></div>
<div class="so_team_logo">
<img alt="" height="84" src="" width="84"/>
</div>
<div class="so_team_name"><span class="so_team_shortname"></span></div>
</div>
</div>
<div class="so_extra_data_2"></div>
<div class="so_place_datas container">
<span></span>, <span></span>
</div>
<div class="so_timeline_widget container clearfix">
<div class="so_timeline_teams">
<div class="so_timeline_team_left clearfix">
<div class="so_team_name"> </div>
<div 

In [11]:

warc_filename = '../../warcs_dir/origo-articles_new3.warc.gz'
# w = WarcCachingDownloader(warc_filename, None, logger, just_cache=True, download_params={'stay_offline': True})

In [12]:
gallery_link.split('/')[4]

'galeria'

In [13]:
with open('gallery_links.txt', 'r') as infile:
    gallery_links = infile.read().splitlines()

In [14]:
set([l.split('/')[3] for l in gallery_links])

{'auto',
 'filmklub',
 'gazdasag',
 'itthon',
 'nagyvilag',
 'sport',
 'tafelspicc',
 'techbazis',
 'tudomany'}

In [15]:
# FROM WARC
_, _, resp = w.get_records(link4)
warc_response_datetime, warc_id, raw_html = extract_resp_record_data(resp)
soup = BeautifulSoup(raw_html, 'lxml')
get_meta(logger, link, soup)


NameError: name 'w' is not defined

In [19]:
get_meta(logger, olimpia_link, from_request(olimpia_link))

2022-03-25 17:05:17,347 WARNING: https://www.origo.hu/sport/olimpia/galeria/20210726-gordeszka-latvanyos-kepek.html GALLERY LINK FAILED TO PARSE


defaultdict(<function html2tei.tei_utils.tei_defaultdict.<locals>.<lambda>()>,
            {'sch:url': 'https://www.origo.hu/sport/olimpia/galeria/20210726-gordeszka-latvanyos-kepek.html',
             'sch:name': 'Az olimpia egyik bemutatkozó sportága egyben az egyik leglátványosabb is - képek',
             'sch:datePublished': datetime.datetime(2021, 7, 26, 0, 0),
             'sch:articleSection': 'Sport',
             'sch:dateModified': datetime.datetime(2021, 7, 26, 0, 0)})

In [17]:
from_request(olimpia_link).find('body', {'class': 'gallery'}).find('header')

<header xmlns:msxsl="urn:schemas-microsoft-com:xslt" xmlns:user="http://mycompany.com/mynamespace"><h1 class="gallery-title">Az olimpia egyik bemutatkozó sportága egyben az egyik leglátványosabb is - képek</h1><a class="go-home" href="#"></a><a class="thumbnails" href="#"></a><a class="full-screen" href="#"></a><a class="close-gallery" href="#"></a></header>

In [18]:
olimpia_link.split('/')[4]

'olimpia'